In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
import tensorflow as tf
from threading import Thread

In [ ]:
# Define the GUI class
class TrashRecycleGUI(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Trash and Recyclable Material Identification")
        self.geometry("800x600")
        self.resizable(False, False)
        self.configure(bg="#f0f0f0")
        self.create_widgets()
        self.predicted_class = None
        self.predicted_probability = None
        self.predicted_material = None
        self.class_names = ["cardboard", "compost", "glass", "metal", "paper", "plastic", "trash"]
        self.class_labels = ["Cardboard", "Compost", "Glass", "Metal", "Paper", "Plastic", "Trash"]
        self.class_colors = ["#0fff00", "#00ff00", "#0000ff", "#ffff00", "#ff00ff", "#000f0f", "#00f000"]
        self.class_colors = [tuple(int(color.lstrip("#")[i:i + 2], 16) / 255 for i in (0, 2, 4)) for color in self.class_colors]
        self.cap = cv2.VideoCapture(0)  # Initialize the camera

    def create_widgets(self):
        # Create a frame to hold the image
        self.image_frame = tk.Frame(self, bg="#f0f0f0")
        self.image_frame.pack(pady=10)

        # Create a label to display the image
        self.image_label = tk.Label(self.image_frame, bg="#f0f0f0")
        self.image_label.pack()

        # Create a frame to hold the predicted class, probability, and material
        self.predicted_frame = tk.Frame(self, bg="#f0f0f0")
        self.predicted_frame.pack(pady=10)

        # Create a label to display the predicted class
        self.predicted_class_label = tk.Label(self.predicted_frame, text="Predicted Class:", bg="#f0f0f0", fg="black")
        self.predicted_class_label.pack(side="left", padx=10)

        # Create a label to display the predicted probability
        self.predicted_probability_label = tk.Label(self.predicted_frame, text="Predicted Probability:", bg="#f0f0f0", fg="black")
        self.predicted_probability_label.pack(side="left", padx=10)

        # Create a label to display the predicted material
        self.predicted_material_label = tk.Label(self.predicted_frame, text="Predicted Material:", bg="#f0f0f0", fg="black")
        self.predicted_material_label.pack(side="left", padx=10)

    def update_camera_feed(self):
        # Get the current frame from the camera
        ret, frame = self.cap.read()

        if ret:
            # Resize the frame to 150x150 pixels
            frame = cv2.resize(frame, (150, 150))

            # Normalize the image array
            image_array = frame / 255.0

            # Reshape the image array to match the input shape of the model
            image_array = np.expand_dims(image_array, axis=0)

            # Make a prediction using the loaded model
            predictions = model.predict(image_array)

            # Get the predicted class and probability
            predicted_class_index = np.argmax(predictions[0])
            predicted_class = self.class_names[predicted_class_index]
            predicted_probability = predictions[0][predicted_class_index]

            # Determine if the predicted material is "Recyclable" or "Trash"
            if predicted_class in ["cardboard", "glass", "metal", "paper", "plastic"]:
                predicted_material = "Recyclable"
                label_color = "#00ff00"  # Green
                label_fg = "green"
            elif predicted_class in ["compost", "trash"]:
                predicted_material = "Trash"
                label_color = "#ff0000"  # Red
                label_fg = "red"
            else:
                predicted_material = "Unknown"
                label_color = "#ffff00"  # Yellow
                label_fg = "yellow"

            # Draw a rectangle around the object with the predicted label
            cv2.rectangle(frame, (10, 10), (140, 140), (int(label_color[1:3], 16), int(label_color[3:5], 16), int(label_color[5:7], 16)), 2)
            cv2.putText(frame, self.class_labels[predicted_class_index], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (int(label_color[1:3], 16), int(label_color[3:5], 16), int(label_color[5:7], 16)), 2)

            # Update the predicted class label, probability label, and material label
            self.predicted_class_label.configure(text=f"Predicted Class: {self.class_labels[predicted_class_index]}", fg=label_fg)
            self.predicted_probability_label.configure(text=f"Predicted Probability: {predicted_probability:.2f}", fg=label_fg)
            self.predicted_material_label.configure(text=f"Predicted Material: {predicted_material}", fg=label_fg)

            # Convert the OpenCV frame to a PIL image
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)
            image = ImageTk.PhotoImage(image)

            # Update the image label with the camera feed
            self.image_label.configure(image=image)
            self.image_label.image = image

        # Schedule the next camera feed update
        self.after(10, self.update_camera_feed)

    def close_camera(self):
        # Release the camera and close the GUI
        self.cap.release()
        self.quit()

# Load the saved model
model = tf.keras.models.load_model("best_model.keras")

# Create the GUI
gui = TrashRecycleGUI()

# Add a button to close the camera and GUI
close_button = tk.Button(gui, text="Close", command=gui.close_camera)
close_button.pack(pady=10)

# Start the GUI event loop
gui.after(0, gui.update_camera_feed)  # Initialize camera feed after the GUI has started
gui.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 525ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 514ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 